## 3/3 周一
经过我的试验 mmdet 应该是可以无痛跑了，问题不大     
我现在要去 HQ-sam 试试这个数据集能不能用    
为什么训练的时候要解耦为两个网络：这样是为了训练时方便，不更新sam参数，只更新新网络参数

mmdetection.    
resume 既加载了模型的权重和优化器的状态，也会继承指定 checkpoint 的迭代次数，不会重新开始训练。load-from 则是只加载模型的权重，它的训练是从头开始的，经常被用于微调模型。其中load-from需要写入配置文件中，而resume作为命令行参数传入。

## 3/4 周二
代码绕来绕去的，真的很晕。 整了一天代码，好像也没整出什么名堂，失败的一天。   
但是我健身了，练肩，还可以，有点累。

## 3/5 周三
从今天开始，都非常关键，，，我要在自己脑海里充斥自己的未来      
思路：既然都很难的话，那我单独创建一个文件夹来微调模型不就不会冲突了，拿出来做隔离，原来的推理，训练都不会受影响。
原来的训练方式不会有问题，那么现在我要试验微调模型会不会有问题，注意学习率要不要调整，，先关注能不能运行起来

微调程序跑起来了，tiny 模型很小，效果也不错
但是发现稍微训练后的模型效果不如已经训练好的官方权重

.eval 把模型设置为评估模式是不会影响模型的梯度更新的，会影响 batch_norm\drop_out 等函数关闭统计学概率。

数据集传上去了，接下来训练，并标更多的数据

## 3/6 周四
上午居然在和师妹讲解红外，想把师妹带入门     
然后一直到现在，还有很多事没有处理呢。  

- 训练起来好吗
- 简历赶紧完善一下

## 3/7 周五
都训练起来了，可视化做的也不错。    
挑选了一部分气体图像大概有60张图片左右，，其实一个场景是可以 多来图像，可能三十张左右，十个场景就有三百张，这样效果差不多可以了
包括有阀门的以及没有阀门的，切换一下。  
训练完好像效果不是特别好，我是不是应该多来点数据，然后用大模型来蒸馏小模型，达到不需要prompt就可以自动分割的效果。

那么我的故事就是，想使用 sam 强大的分割性能，但是 zero-shot 的方式在特定领域效果还是不行，于是我用带 prompt 的大模型来蒸馏不带 prompt 的小模型实现专有效果。

无形物体的自动实例分割，包括气体烟雾，在红外光下更明显，但是任务却是相当有挑战性的。

---多几轮训练并不能解决我的问题      
- 多数据
- 模型改为无prompt的方式 最好使用扩散模型的解码器

## 3/8 周六
看了 RCG 里面很很多扩散模型的知识

首先是预训练好的无监督模型 Moco 提取表征表示。   
然后是 RDM 扩散模型噪声还原表征，现在还不知道是干嘛的      
最后图像生成模型，还是晕晕的

## 3/9 周日
重温研究一下 RCG ，我发现了问题所在：    
图像生成器的训练阶段和推理阶段是不同的：在**训练**阶段直接以图片为输入，可以通过编码器轻松得到表征；**推理**时候以噪声作为输入，表征只能通过RDM得到。   
扩散模型 RDM 的训练和推理阶段是不是也不一样：**训练**以添加噪声的表征作为输入；**推理**以噪声作为输入还原表征。

扩散模型在训练时通过对图片添加噪声将其变成纯噪声，训练过程反向去噪恢复图像。推理时直接从噪声中采样扩散生成图像，若无控制条件，无法控制生成图像。      

DDPM DDIM

## 3/11 周二
- $q$ 代表 **前向加噪过程**，用于训练时添加噪声。     
- $p$ 代表 **反向去噪过程**，用于生成时去除噪声。 

学习 DDPM 源码，明天我将直接查看代码讲解！！

## 3/12 周三
突然觉得时间真的不多了，找工作迫在眉睫，我真的还有机会吗，真的是难受。现在要做的是打起精神抓紧学习。    

时间步$t$噪声参数：$\beta_{t}$     
衰减因子：$\alpha_{t} = 1 - \beta_{t}$     
累积衰减因子：$\hat{\alpha_{t}} = \prod\limits_{s=1}^{t} \alpha_{s}$     
从任意时间步$t$计算带噪声的数据：$x_{t} = \sqrt{\hat{\alpha_{t}}} x_{0} + \sqrt{1-\hat{\alpha_{t}}} \epsilon$


## 3/13 周四
扩散模型，行动！！！记了几个公式并不能说明什么

- **训练时**直接根据原始图像$x0$和时间步t预测添加到$xt$上的噪声，以真实噪声作为真值计算损失；    
- **推理时**以随机噪声作为$xt$，预测噪声反推$x0$，计算$xt-1$，一步一步反向扩散得到最终的$x0$。

ps: 在扩散模型中，虽然可以直接从第 $t$ 步的噪声图像 $x_{t}$ 预测原始无噪声图像 $x_{0}$ ，但反推时仍然需要逐步从 $x_{t}$ 预测 $x_{t-1}$ 逐渐去噪得到最终无噪图像 $x_{0}$ ？   
---逐步去噪有更强的稳定性，每一步的噪声水平较低更好预测；逐步去噪过程更符合马尔可夫链的连续表达；逐步去噪过程使得生成更加可控。


赶快回忆一下我的图像分割：用 hq-sam 微调模型，使用 vit-tiny 作为 image_encoder 模型参数量很小，训练过后好像模型效果不是特别好，还没[**建立数据集**]，还没[**修改模型**]。     
分割模型先放一放，转向研究扩散模型，，，我要扩充自己的简历，安排好事，，，，真的要为自己的未来考虑了！！！！

## 3/14 周五
早上醒晚了，但是收到消息论文意见回来了，从结果来看的话，是比较乐观的。小修，说不定下次直接接收了。         
天大的好消息，但是此刻我还是有点不知所措,,,开干好吗！！！

研究了半天意见，应该是比较好改的；下午改半天简历，也改了一半，没完全改完，缺一部分，差扩散模型这一部分；

## 3/15 周六
可以直接通过模型的配置文件查看模型的总体结构。    
看清了模型的总体结构，是一个直接的全连接结构，没有高宽的变化。     

看了模型的数据读取程序，用的是经典函数 **torch.utils.data.DataLoader** 和 **datasets.ImageFolder** 得到批量的图像和数据对。     
没有transformer的介入，那么高宽是没有那么重要的，下载一个手写数字数据集。

今天直接训练呗，，，直接训起来，，，，扩散模型 DDPM ！！！


## 3/17 周一


```bash
# HQ-sam
torchrun --nproc_per_node=4 train.py --checkpoint ./pretrained_checkpoint/sam_vit_b_01ec64.pth --model-type vit_b --output work_dirs/hq_sam_b

CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc_per_node=2 train.py --checkpoint ./pretrained_checkpoint/sam_vit_b_01ec64.pth --model-type vit_b --output work_dirs/hq_sam_b

CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc_per_node=2 fine_tune.py

# mmdet
CUDA_VISIBLE_DEVICES=0,1 ./tools/dist_train.sh configs/gas_det/mask_rcnn_r50.py 2

# DiT
CUDA_VISIBLE_DEVICES=2,3 torchrun --nnodes=1 --nproc_per_node=2 train.py --model DiT-L/4 --data-path /home/2301062/data/cat_12 --num-classes 12
``` 

```bash
# 服务器走本地代理终于突破了
# IP 端口 直接上苹果设置里查询
export http_proxy="http://172.29.55.53:7897"    
export https_proxy="http://172.29.55.53:7897"
```